In [2]:
#importation des bibliothèques nécessaires
import selenium
import re
from selenium import webdriver

In [3]:
def recup_bdd(id_chaine) :
    
    #chaine = chaine.replace(' ' , '')
    #initialisation webdriver
    chrome = webdriver.Chrome(executable_path="./scrap_playlist/chromedriver")
    
    #récupération des données
    bdd = []
    lien = 'https://www.youtube.com/channel/{}/playlists?view=1&sort=dd&shelf_id=0'.format(id_chaine)
    chrome.get(lien)
    chrome.refresh()
    tag = chrome.find_elements_by_css_selector('a')
    l_url = []
    l_title = []
    for a in tag:
        url = a.get_attribute('href')
        try:
            if url.startswith("https://www.youtube.com/playlist?"):
                l_url.append(url)
            if url.startswith("https://www.youtube.com/watch?"):
                l_title.append(a.get_attribute('title'))
        except:
            pass

    l_title = list(filter(None,l_title))

    for idx,i in enumerate(l_url) :
        
            chrome.get(i)
            tag = chrome.find_element_by_id("stats")
            res = []
            for j in tag.find_elements_by_tag_name('span'):
                res.append(j.text)
                
            desc = chrome.find_element_by_id('description')
            description = desc.find_elements_by_tag_name('yt-formatted-string')[0].text
            
            #Obtenier
            img = chrome.find_elements_by_css_selector('img')[0].get_attribute('src')
            
            
            a = chrome.find_elements_by_class_name('style-scope ytd-thumbnail-overlay-time-status-renderer')
            p = [z.text for z in a]
        
            if len(res)>0:
                nbr_videos = res[0]
                if len(res)==5:
                    last_modif = res[2]+res[3]+res[4]
                    last_modif = last_modif.replace('Mise à jour il y a','')

                if len(res)==4:
                    last_modif = res[2]+res[3]
                    last_modif = last_modif.replace('Dernière modification le','')
                
                if len(res)==2:
                    nbr_videos = 0
                    last_modif = res[1]
            else:
                #Si une video la liste est vide
                nbr_videos = 1
                
            vues = chrome.find_element_by_xpath("//yt-formatted-string[2]").text
            vues = vues.replace('\u202f','')
            id_playlist = re.findall(r'https:\/\/www\.youtube\.com\/playlist\?list=([a-zA-Z0-9-_]*)',i)[0]

            bdd.append({ '_id' : id_playlist,
                    'id_youtubeur' : id_chaine,
                    'playlist' : l_title[idx],
                    'nbr_videos' : nbr_videos,
                    'derniere_MAJ' : last_modif,
                    'vues' : vues,
                    'url_img' : img,
                    'description': description,
                    'video_n': list(filter(None,p))
                   })
    chrome.quit()
    return bdd

In [4]:
test = recup_bdd('UC33rpHD9QGwsCEUPU_mrh1A')

In [5]:
test[0]

{'_id': 'PLQut5OXpV-0gk9CcjfInV9hNIEAld-Tpp',
 'id_youtubeur': 'UC33rpHD9QGwsCEUPU_mrh1A',
 'playlist': 'AWIPS',
 'nbr_videos': '3',
 'derniere_MAJ': '13 juil. 2020',
 'vues': '41 vues',
 'url_img': 'https://i.ytimg.com/vi/hjytnJOxrOg/hqdefault.jpg?sqp=-oaymwEXCNACELwBSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBhOFcrwPvTwTh4w0qYOdTlyKa0Rw',
 'description': '',
 'video_n': ['4:19', '0:20', '0:20']}